## Whale sounds
this notebook makes you make whale sounds. 

In [16]:
# first, imports. 
# cv2 and mediapipe: mouth detectionp=
import cv2
import mediapipe as mp
# numpy 4 arrays :P
import numpy as np
from collections import deque
import time
# for the whale sound modulation!
import sounddevice as sd
import librosa
from scipy.signal import convolve
from pedalboard import Pedalboard, Reverb, Chorus, Delay, PitchShift, LowShelfFilter, Gain, LowpassFilter

# to show face
from IPython.display import display, clear_output

We start by loading a whale sound.

In [17]:
# load all sounds
bases = ['balene', 'hd', 'hump']
balene_do_hi, sr = librosa.load('sounds/balene-do (high).wav')
balene_re, _ = librosa.load('sounds/balene-re.wav')
balene_mi, _ = librosa.load('sounds/balene-mi (3).wav')
balene_fa, _ = librosa.load('sounds/balene-fa.wav')
balene_sol, _ = librosa.load('sounds/balene-sol (5).wav')
balene_la, _ = librosa.load('sounds/balene-la.wav')
balene_ti, _ = librosa.load('sounds/balene-ti.wav')
balene_do, _ = librosa.load('sounds/balene-low-do.wav')

hd_do_hi, sr = librosa.load('sounds/hd-hi.wav')
hd_re, _ = librosa.load('sounds/hd-re.wav')
hd_mi, _ = librosa.load('sounds/hd-mi.wav')
hd_fa, _ = librosa.load('sounds/hd-fa.wav')
hd_sol, _ = librosa.load('sounds/hd-so.wav')
hd_la, _ = librosa.load('sounds/hd-la.wav')
hd_ti, _ = librosa.load('sounds/hd-ti.wav')
hd_do, _ = librosa.load('sounds/hd-do.wav')

hump_do_hi, sr = librosa.load('sounds/hump-hido.wav')
hump_re, _ = librosa.load('sounds/hump-re.wav')
hump_mi, _ = librosa.load('sounds/hump-mi.wav')
hump_fa, _ = librosa.load('sounds/hump-fa.wav')
hump_sol, _ = librosa.load('sounds/hump-sol.wav')
hump_la, _ = librosa.load('sounds/hump-la.wav')
hump_ti, _ = librosa.load('sounds/hump-ti.wav')
hump_do, _ = librosa.load('sounds/hump-do.wav')

# make high, mid, and low arrays!
hump = [hump_do, hump_re, hump_mi, hump_fa, hump_sol, hump_la, hump_ti, hump_do_hi]
hd = [hd_do, hd_re, hd_mi, hd_fa, hd_sol, hd_la, hd_ti, hd_do_hi]
balene = [balene_do, balene_re, balene_mi, balene_fa, balene_sol, balene_la, balene_ti, balene_do_hi]

hi_hump = [librosa.effects.pitch_shift(sound, sr=sr, n_steps=12) for sound in hump]
hi_hd = [librosa.effects.pitch_shift(sound, sr=sr, n_steps=12) for sound in hd]
hi_balene = [librosa.effects.pitch_shift(sound, sr=sr, n_steps=12) for sound in balene]

lo_hump = [librosa.effects.pitch_shift(sound, sr=sr, n_steps=-12) for sound in hump]
lo_hd = [librosa.effects.pitch_shift(sound, sr=sr, n_steps=-12) for sound in hd]
lo_balene = [librosa.effects.pitch_shift(sound, sr=sr, n_steps=-12) for sound in balene]

#sd.play(hump_do)

Now, we make out sound sequence. 


In [18]:

sounds = [
    hump[0],
    lo_hd[0],
    hump[2],
    lo_hump[4],
    lo_balene[1],
    hd[4],
    lo_hd[-1],
    lo_hump[-3],
    hump[1],
    hump[3],
    lo_balene[7],
    hd[0],
    hd[2],
    lo_hump[4],
    hi_hd[-1],
    hi_hd[4],
    hd[3],
    balene[0],
    lo_balene[-1],
    hump[3],
    hi_hump[-1],
    hi_hump[-3],
    hd[-1],
    hi_hump[-5],
    hump[2],
    lo_hump[4],
    lo_balene[1],
    hd[4],
    lo_hd[-1],
    lo_hump[-3],
    hump[1],
    hump[3],
    lo_balene[7],
    lo_hump[-3],
    hump[1],
    hump[3],
    lo_balene[7],
    hd[0],
    hd[2],
    lo_hump[4],
    hi_hd[-1],
    hi_hd[4],
    hd[3],
    balene[0],
    lo_balene[-1],
    hump[3],
    hi_hump[-1],
]


We can apply effects using pedalboard. 

In [19]:
board = Pedalboard([Reverb(room_size=0.95, damping=1, wet_level=0.8), 
                    # Delay(delay_seconds=0.5, feedback=1),
                    LowShelfFilter(),
                    LowpassFilter(),
                    Gain(gain_db=3)
                    ])

sounds = [board(a.astype('float32'), sr) for a in sounds]
sounds = [librosa.effects.time_stretch(sound.astype('float32'), rate=0.6) for sound in sounds]

In [20]:
sd.play(sounds[1])

Now, create a function to alter the pitch given a number of semitones.

Yay! Time to play with the mouth processing. 

In [21]:
'''
Distance: determine distance between 2 points
'''
def distance(a, b):
    return np.linalg.norm(a - b)

In [22]:
!tccutil reset Camera


Successfully reset Camera


In [23]:
# create a face mesh from mp
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(refine_landmarks=True)
# create a video capture object
cap = cv2.VideoCapture(0)
print("Opened:", cap.isOpened())

Opened: False


I0000 00:00:1764772703.705424 6800017 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M4 Pro
W0000 00:00:1764772703.708112 6801198 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1764772703.714916 6801194 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
OpenCV: not authorized to capture video (status 0), requesting...
OpenCV: camera failed to properly initialize!


Now that the camera and mouth register have been set up, we can begin processing words. 

In [24]:
# reset stream!! or it will get MAD!!!
stream = None

In [25]:
# hyperparams
mouth_open = False
mouth_open_prev = False
current_pitch = 0      

# eventually, we will be randomizing the semitone changes + the sound fx
step_size = 2 

# sound params
sound_index = 0
fs = 44100      

# apply cooldown time --> stops sound from turning evil bc of too much hap
cooldown_time = 2  
#track lsast sound
last_trigger_time = 0 

# mixing param
buffer_size = 512
active_sounds = deque()

In [26]:

# add the new sound to the active songs.
def trigger_sound(sound_array):
    active_sounds.append((sound_array, 0))

# gives a limit so we dont have evil soudns
def soft_limit(x):
    return np.tanh(x)

def audio_callback(outdata, frames, time, status):
    if status:
        print(status)

    mixed = np.zeros(frames, dtype=np.float32)
    still_playing = deque()

    # mix
    for sound_array, idx in active_sounds:
        end_idx = min(idx + frames, len(sound_array))
        chunk_len = end_idx - idx
        if chunk_len > 0:
            mixed[:chunk_len] += sound_array[idx:end_idx] * 0.5   # reduce gain
        if end_idx < len(sound_array):
            still_playing.append((sound_array, end_idx))

    # apply soft limiting
    mixed = soft_limit(mixed)

    outdata[:] = mixed.reshape(-1, 1)

    active_sounds.clear()
    active_sounds.extend(still_playing)


In [27]:
# restart the stream if it exists!! so everythign doesnt crash.
if stream is not None:
    try:
        stream.stop()
        stream.close()
    except:
        pass


# create stream object
stream = sd.OutputStream(
    channels=1,
    #samplerate=fs,
    blocksize=buffer_size,
    callback=audio_callback,
)
stream.start()

In [28]:
# Run loooooooop
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # create face mesh
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(rgb)

    if results.multi_face_landmarks:
        # find face landmarks
        h, w, _ = frame.shape
        face = results.multi_face_landmarks[0]
        upper = np.array([face.landmark[13].x * w, face.landmark[13].y * h])
        lower = np.array([face.landmark[14].x * w, face.landmark[14].y * h])

        # create circles on lips
        cv2.circle(frame, tuple(upper.astype(int)), 3, (0, 255, 0), -1)
        cv2.circle(frame, tuple(lower.astype(int)), 3, (0, 0, 255), -1)

        #calculate mouth dsitance and time
        mouth_dist = np.linalg.norm(upper - lower)
        now = time.time()
        
        # when mouth is open:
        if mouth_dist > 15:
            if not mouth_open and (now - last_trigger_time >= cooldown_time):
                mouth_open = True
                y_sound = sounds[sound_index % len(sounds)]
                
                # trigger sound!!
                trigger_sound(y_sound)
                sound_index += 1 # WANT TO MAKE This RANDOM!!!
                
                last_trigger_time = now 
                
        # close mouth :P
        else:
            mouth_open = False

    False
    cv2.imshow("cutting in", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

In [29]:
stream.stop()
stream.close()
cap.release()
cv2.waitKey(1)
cv2.destroyAllWindows()


In [30]:
import os
import soundfile as sf  # pip install soundfile

# Folder to save web-ready whale sounds
out_dir = "web_whale_sounds"
os.makedirs(out_dir, exist_ok=True)

# 'sounds' is your final processed list of numpy arrays from the notebook
for i, sound in enumerate(sounds):
    # Ensure mono float32
    y = sound.astype("float32").flatten()
    filename = os.path.join(out_dir, f"whale_{i:02d}.wav")
    sf.write(filename, y, sr)  # 'sr' is your sampling rate from librosa.load
    print("wrote", filename)

wrote web_whale_sounds/whale_00.wav
wrote web_whale_sounds/whale_01.wav
wrote web_whale_sounds/whale_02.wav
wrote web_whale_sounds/whale_03.wav
wrote web_whale_sounds/whale_04.wav
wrote web_whale_sounds/whale_05.wav
wrote web_whale_sounds/whale_06.wav
wrote web_whale_sounds/whale_07.wav
wrote web_whale_sounds/whale_08.wav
wrote web_whale_sounds/whale_09.wav
wrote web_whale_sounds/whale_10.wav
wrote web_whale_sounds/whale_11.wav
wrote web_whale_sounds/whale_12.wav
wrote web_whale_sounds/whale_13.wav
wrote web_whale_sounds/whale_14.wav
wrote web_whale_sounds/whale_15.wav
wrote web_whale_sounds/whale_16.wav
wrote web_whale_sounds/whale_17.wav
wrote web_whale_sounds/whale_18.wav
wrote web_whale_sounds/whale_19.wav
wrote web_whale_sounds/whale_20.wav
wrote web_whale_sounds/whale_21.wav
wrote web_whale_sounds/whale_22.wav
wrote web_whale_sounds/whale_23.wav
wrote web_whale_sounds/whale_24.wav
wrote web_whale_sounds/whale_25.wav
wrote web_whale_sounds/whale_26.wav
wrote web_whale_sounds/whale